<a href="https://colab.research.google.com/github/Shweta-afk/generativeAI-poemgenerator/blob/main/Poem_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



**Installing pandas**:

In the provided code, pandas is used to read the poetry dataset from a CSV file

**Installing transformers**:

**Using the GPT2Tokenizer:** to tokenize the text for the GPT2 model.
Using the GPT2

# **INSTALL ALL THE LIBRARIES BEFORE RUNNING THE CODE**

In [ ]:
pip install pandas


In [6]:
!pip install -U accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 7.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [7]:
!pip install transformer

ERROR: Could not find a version that satisfies the requirement transformer (from versions: none)
ERROR: No matching distribution found for transformer


In [8]:
pip install torch

In [9]:

!pip install -U transformers[torch]


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 402.6/402.6 kB 30.1 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.40.1
    Uninstalling transformers-4.40.1:
      Successfully uninstalled transformers-4.40.1


This code is the read the dataset provided and to display few rows of the dataset. And to convert the "Poem" column to a list.

In [ ]:
import pandas as pd

# Loading the dataset
df = pd.read_csv('poemfeed.csv')
texts = df['Poem'].tolist()

# Display the first few rows of the DataFrame to verify it's loaded correctly
print(df.head())


In [ ]:
import pandas as pd
from transformers import GPT2Tokenizer

df = pd.read_csv('poemfeed.csv')

#Faced an error because of the empty blocks in the poem coloumn
# Handle missing data in the 'Poem' column
df.fillna({'Poem': ''}, inplace=True)  # Replace NaN values in 'Poem' with empty strings

texts = df['Poem'].tolist()

tokenizer = GPT2Tokenizer.from_pretrained('gpt2-medium')
tokenized_texts = [tokenizer.encode(text, add_special_tokens=True, max_length=512, truncation=True) for text in texts]

# Verify tokenized texts
print(tokenized_texts[:3])


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

class PoetryDataset(Dataset):
    def __init__(self, tokenized_texts):
        self.tokenized_texts = tokenized_texts

    def __len__(self):
        return len(self.tokenized_texts)

    def __getitem__(self, idx):
        return torch.tensor(self.tokenized_texts[idx])

dataset = PoetryDataset(tokenized_texts)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)


In [ ]:
from transformers import GPT2LMHeadModel, Trainer, TrainingArguments

model = GPT2LMHeadModel.from_pretrained('gpt2-medium')

training_args = TrainingArguments(
    output_dir='./results',
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
    logging_dir='./logs',
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
)

trainer.train()


## **Main Code**

In [2]:

from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd

from google.colab import userdata
userdata.get('PersonalProjects')

 # Load and Prepare the Dataset
df = pd.read_csv('poemfeed.csv')
texts = df['Poem'].tolist()

#Tokenize the Dataset
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-medium')
# Handle missing data in the 'Poem' column
df.fillna({'Poem': ''}, inplace=True)
# Replace NaN values in 'Poem' with empty strings
texts = df['Poem'].tolist()

tokenizer = GPT2Tokenizer.from_pretrained('gpt2-medium')
tokenized_texts = [tokenizer.encode(text, add_special_tokens=True,
                                    max_length=512,
                                    truncation=True) for text in texts]



#Create a PyTorch Dataset
class PoetryDataset(Dataset):
    def __init__(self, tokenized_texts):
        self.tokenized_texts = tokenized_texts

    def __len__(self):
        return len(self.tokenized_texts)

    def __getitem__(self, idx):
        # Create attention mask
        attention_mask = [1] * len(self.tokenized_texts[idx])

        return {'input_ids': torch.tensor(self.tokenized_texts[idx]),
                'attention_mask': torch.tensor(attention_mask)}

dataset = PoetryDataset(tokenized_texts)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

#Fine-Tune the Model
model = GPT2LMHeadModel.from_pretrained('gpt2-medium')

training_args = TrainingArguments(
    output_dir='./results',
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
    logging_dir='./logs',
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
)


#Generate Poems
prompt = "your lie in april that i still remember"
inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(inputs['input_ids'],
                         attention_mask=inputs['attention_mask'],
                         max_length=100,
                         num_return_sequences=1,
                         pad_token_id=tokenizer.eos_token_id,
                         no_repeat_ngram_size=2,
                         temperature = 0.8)

generated_poem = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_poem)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


your lie in april that i still remember you were there. i was so happy to see you again. you are so sweet and so kind. thank you for everything.

I am so sorry for the mess i made. I am sorry that you had to go through this. You are my hero. Thank you so much for all the love and support.
